In [1]:
from agently_triggerflow import TriggerFlow

flow = TriggerFlow()

@flow.chunk
def user_input(trigger_data):
    print("Some initial data:", trigger_data)
    return input("Input something:")

@flow.chunk
def echo(trigger_data):
    print("You just input:", trigger_data)
    flow.data.set("trigger", trigger_data)

@flow.chunk
def triggered(trigger_data):
    print("I've been triggered")

flow.chunk(user_input).then(echo)
flow.on_data("trigger").then(triggered)

flow.start(user_input, "Let the show begin.")
pass

Some initial data: Let the show begin.
You just input: Agently Stage
I've been triggered


In [2]:
from agently_triggerflow import TriggerFlow

flow = TriggerFlow()

@flow.chunk
def count(_):
    counter = flow.data.get("counter", 0) + 1
    print("Times:", counter)
    flow.data.set("counter", counter)

@flow.chunk
def user_input(trigger_data):
    return input("Exit[Y/N]:")

(
    flow.wait(user_input)
        .if_condition(lambda trigger_data: trigger_data.lower() != "y")
            .then(count)
            .then(user_input)
)

flow.start(user_input)
pass

Times: 1
Times: 2
[Assistant]: 

In [6]:
#!pip install Agently==3.4.1.2
import Agently
from agently_triggerflow import TriggerFlow

flow = TriggerFlow()

agent = (
    Agently.create_agent()
        .set_settings("current_model", "OAIClient")
        .set_settings("model.OAIClient.auth", { "api_key": "<Your-API-Key>" })
        .set_settings("model.OAIClient.url", "https://api.deepseek.com/v1")
        .set_settings("model.OAIClient.options", { "model": "deepseek-reasoner" })
)

@flow.chunk
def user_input(trigger_data):
    return input("[User]:")

@flow.chunk
def agent_response(trigger_data):
    response = (
        agent
            .input(trigger_data)
            .get_delta_generator()
    )
    for item in response:
        flow.result.put(item)
    flow.result.put_stop()

flow.chunk(user_input).to(agent_response)
response = flow.start()
print("[Assistant]: ", end="")
for delta in response:
    print(delta, end="", flush=True)

[Assistant]: Let \( D \) be the number of donkeys and \( S \) be the number of snails. We know the following:

1. The total number of heads is 20:
   \[
   D + S = 20
   \]
2. The total number of legs is 36. Donkeys have 4 legs each, and snails have 0 legs each:
   \[
   4D + 0S = 36
   \]

From the second equation, we can solve for \( D \):
   \[
   4D = 36 \implies D = 9
   \]

Substituting \( D = 9 \) into the first equation:
   \[
   9 + S = 20 \implies S = 11
   \]

Thus, there are \(\boxed{9}\) donkeys and \(\boxed{11}\) snails in the cage.

In [ ]:
import Agently
from agently_triggerflow import TriggerFlow

flow = TriggerFlow()

agent = (
    Agently.create_agent()
        .set_settings("current_model", "ERNIE")\
        .set_settings("model.ERNIE.auth", {
            "aistudio": "<You-ERNIE-Access-Token>",
        })
        .set_settings("model.ERNIE.options", { "model": "ernie-speed" })
)

@flow.chunk
def user_input(trigger_data):
    return input("[User]:")

@flow.chunk
def agent_response(trigger_data):
    response = (
        agent
            .chat_history(flow.data.get("chat_history"))
            .input(trigger_data)
            .get_delta_generator()
    )
    reply = ""
    print("[Assistant]: ", end="")
    for delta in response:
        reply += delta
        print(delta, end="", flush=True)
    print("")
    return reply

@flow.chunk
def add_chat_history(trigger_data):
    chat_history = flow.data.get("chat_history", [])
    chat_history.append({ "role": "user", "content": trigger_data["user_input"] })
    chat_history.append({ "role": "assistant", "content": trigger_data["agent_response"] })
    flow.data.set("chat_history", chat_history)

(
    flow
        .wait(user_input)
        .if_condition(lambda trigger_data: trigger_data != "#exit")
            .to(agent_response)
        .else_condition()
            .to(flow.END)
)
(
    flow
        .wait_all(user_input, agent_response)
            .to(add_chat_history)
            .to(user_input)
)

flow.start()
pass

[WARNING][2025-02-07 15:55:54.391] redis_rate_limiter.py:21 [t:8311833152]: No redis installed, RedisRateLimiter unavailable. Ignore this warning if you don't need to use qianfan SDK in distribution environment


[Assistant]: Hello there! It's a pleasure to have a conversation with you. I hope you're having a great day so far. Feel free to share anything that you would like to discuss.
[Assistant]: I have reminded you to buy a dozen eggs from the market. Please don't forget to buy them when you go to the market.
[Assistant]: I have also reminded you to buy 6 boxes of milk. Please don't forget to buy them together with the eggs when you go to the market.
[Assistant]: Just now we reminded you to buy some items at the market, including a dozen eggs and six boxes of milk.
[Assistant]: Sure, here is a list of the items you need to buy at the market:

1. A dozen eggs
2. 6 boxes of milk

Please make sure to buy all the items on the list when you go to the market. Do you need any help with anything else?


In [2]:
# Stress Testing
import time
import asyncio
import random
from agently_stage import Stage
from agently_triggerflow import TriggerFlow

flow = TriggerFlow(name="test", max_workers=100)

counter = 0

@flow.chunk
def task_sender(trigger_data):
    for item in trigger_data:
        flow.data.set("item", item)
    flow.data.set("item", StopIteration)

@flow.chunk
async def item_consumer(trigger_data):
    # Simulate async or sync delay
    await asyncio.sleep(1 / random.randint(1, 100))
    #time.sleep(1 / random.randint(1, 100))
    print("Received:", trigger_data)
    global counter
    counter += 1
    print("Now:", counter)

flow.chunk(task_sender)
flow.on_data("item").to(item_consumer)
flow.collect_data("item")

response = flow.start(task_sender, [item for item in range(1, 10000)])

Received: 7
Now: 1
Received: 3
Now: 2
Received: 4
Now: 3
Received: 9
Now: 4
Received: 8
Now: 5
Received: 6
Now: 6
Received: 5
Now: 7
Received: 2
Now: 8
Received: 1
Now: 9
Received: 11
Now: 10
Received: 10
Now: 11
Received: 12
Now: 12
Received: 21
Now: 13
Received: 33
Now: 14
Received: 24
Now: 15
Received: 74
Now: 16
Received: 171
Now: 17
Received: 43
Now: 18
Received: 91
Now: 19
Received: 234
Now: 20
Received: 83
Now: 21
Received: 313
Now: 22
Received: 149
Now: 23
Received: 322
Now: 24
Received: 23
Now: 25
Received: 41
Now: 26
Received: 258
Now: 27
Received: 185
Now: 28
Received: 151
Now: 29
Received: 195
Now: 30
Received: 286
Now: 31
Received: 246
Now: 32
Received: 156
Now: 33
Received: 388
Now: 34
Received: 268
Now: 35
Received: 481
Now: 36
Received: 31
Now: 37
Received: 435
Now: 38
Received: 154
Now: 39
Received: 493
Now: 40
Received: 457
Now: 41
Received: 353
Now: 42
Received: 389
Now: 43
Received: 476
Now: 44
Received: 525
Now: 45
Received: 482
Now: 46
Received: 143
Now: 47
Receiv